In [75]:
%%capture
!pip install halftone

In [2]:
# importing image object from PIL
import math
from PIL import Image, ImageDraw, ImageStat
import halftone as ht
import random
import numpy as np
import os
import imageio

### №1.

In [10]:
# Adaption of answer https://stackoverflow.com/a/10575940/355230
def halftone(img, sample, scale, angle=45):
    ''' Returns a halftone image created from the given input image `img`.
     -> `sample` (in pixels), determines the sample box size from the original
    image. The maximum output dot diameter is given by `sample` * `scale`
    (which is also the number of possible dot sizes). So `sample` == 1 will
    preserve the original image resolution, but `scale` must be > 1 to allow
    variations in dot size.
    '''
    img_grey = img.convert('L')  # Convert to greyscale.
    channel = img_grey.split()[0]  # Get grey pixels.
    channel = channel.rotate(angle, expand=1)
    size = channel.size[0]*scale, channel.size[1]*scale

    bitmap = Image.new('1', size)
    draw = ImageDraw.Draw(bitmap)

    for x in range(0, channel.size[0], sample):
        for y in range(0, channel.size[1], sample):
            box = channel.crop((x, y, x+sample, y+sample))
            mean = ImageStat.Stat(box).mean[0]
            diameter = (mean/255) ** 0.5
            edge = 0.5 * (1-diameter)
            x_pos, y_pos = (x+edge) * scale, (y+edge) * scale
            box_edge = sample * diameter * scale
            draw.ellipse((x_pos, y_pos, x_pos+box_edge, y_pos+box_edge),
                         fill=255)

    bitmap = bitmap.rotate(-angle, expand=1)
    width_half, height_half = bitmap.size
    xx = (width_half - img.size[0]*scale) / 2
    yy = (height_half - img.size[1]*scale) / 2
    bitmap = bitmap.crop((xx, yy, xx + img.size[0]*scale,
                                  yy + img.size[1]*scale))
    return Image.merge('1', [bitmap])

def Luminance (R,G,B):
  return (0.212*R) + (0.7152*G) + (0.0722*B)

In [73]:
w,h = 1000,1000
mu, sigma = 255/2, 255


pixels = np.random.normal(mu, sigma,size=(w,h,3))
image = Image.fromarray(pixels.astype('uint8'), 'RGB')
image_file = image.convert('1') # convert image to black and white
image_file.save('your_file.png')
image.save("your_file_RGB.png")

im = Image.open("your_file.png") 

data = np.array(im.getdata())
pixels_from_image = data.reshape(*im.size, -1)


In [74]:
width_i = 5
brightness = 0
start_x  = 0
while True:
  print(f"w = {w}; brightness = {brightness}; start_x = {start_x}")
  pixels_from_image[:,start_x : start_x + width_i ] = pixels_from_image[:,start_x : start_x + width_i ] * brightness
  brightness = width_i/w
  start_x += width_i
  width_i *=2
  if start_x > w:
    break


w = 1000; brightness = 0; start_x = 0
w = 1000; brightness = 0.005; start_x = 5
w = 1000; brightness = 0.01; start_x = 15
w = 1000; brightness = 0.02; start_x = 35
w = 1000; brightness = 0.04; start_x = 75
w = 1000; brightness = 0.08; start_x = 155
w = 1000; brightness = 0.16; start_x = 315
w = 1000; brightness = 0.32; start_x = 635


In [71]:
pixels_from_image = pixels_from_image.reshape((w, h)).astype('uint8')*255
image = Image.fromarray(pixels_from_image.astype('uint8'))
image.save("your_file_fixed.png")

### №2.

In [4]:
frequencies = {
    "red":430*1e12,
    "orange":492*1e12,
    "yellow":520*1e12,
    "green":575*1e12,
    "cyan":610*1e12,
    "blue":645*1e12,
    "violet":730*1e12
}

def phases_waves( x,y,t,
                 x0:float=0,y0:float=0,
                 frequency_wave:str="red", 
                 Amplitude:float=20,
                 phi_0=0):
  C = 299792458.0 # "Speed of light in vacuum
  freq = frequencies[frequency_wave]

  lambda_wave = C/freq
  k = 2 * np.pi / lambda_wave
  return Amplitude * np.cos(
      (((x-x0)**2 + (y-y0)**2)*k)**0.5 - 2 * np.pi * freq * t + phi_0
  )
  vphases_waves = np.vectorize(phases_waves)

In [5]:
!pwd

/content


In [6]:
def clear_folder(folder):
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

In [8]:
import os
WAVES_IMAGES = "/content/WAVES_jpgs"
if not os.path.exists(WAVES_IMAGES):
    os.makedirs(WAVES_IMAGES)
WAVES_GIFS = "/content/WAVES_gifs"
if not os.path.exists(WAVES_GIFS):
    os.makedirs(WAVES_GIFS)

w,h = 500,500

xc0,yc0 = 0,0
xc1,yc1 = w,0

#waves = ["red","orange","yellow","green","cyan","blue","violet"]
Amplitudes = range(50,250,50)
T = 100
# for wave in waves:
wave = 'red'
for Amplitude in Amplitudes:
  images = []
  for t_i in np.linspace(0,T,3*T):
    X = np.tile(np.linspace(0,w,w), (w, 1))
    Y = np.tile(np.linspace(0,h,h).reshape((-1,1)), (1, h))
    pixels = np.full((w,h,3), 255)
    pixels[:,:,0] = pixels[:,:,1] = pixels[:,:,2] = phases_waves(
        t=t_i,x=X,y=Y,x0=xc0,y0=yc0,frequency_wave=wave,Amplitude=Amplitude
        ) + phases_waves(
            t=t_i,x=X,y=Y,x0=xc1,y0=yc1,frequency_wave=wave,Amplitude=Amplitude
            )

    image = Image.fromarray(pixels.astype('uint8'), 'RGB')
    images.append(image)
    image.save(f"{WAVES_IMAGES}/{t_i}.jpeg")
    
  imageio.mimsave(f'{WAVES_GIFS}/{wave.capitalize()}-{Amplitude}-(({xc0}_{yc0})_({xc1}_{yc1})).gif', images)
  clear_folder(WAVES_IMAGES)
# change brightness of vertical lines


!zip -r {WAVES_GIFS}.zip {WAVES_GIFS}


  adding: content/WAVES_gifs/ (stored 0%)
  adding: content/WAVES_gifs/Red-50-((0_0)_(500_0)).gif (deflated 9%)
  adding: content/WAVES_gifs/Red-200-((0_0)_(500_0)).gif (deflated 7%)
  adding: content/WAVES_gifs/Red-150-((0_0)_(500_0)).gif (deflated 7%)
  adding: content/WAVES_gifs/Red-100-((0_0)_(500_0)).gif (deflated 7%)


In [ ]:
print(np.cos(1))
print(np.cos(1 + 2*np.pi/360*5))

0.5403023058681398
0.46490726379104985


In [ ]:
import os, shutil
folders = [WAVES_IMAGES, WAVES_GIFS]
for folder in folders:
  clear_folder(folder)

In [ ]:
w,h = 500,500
N = int(input("Количество источников: "))
R = float(input("Радиус круговой траектории: "))
assert N > 0, "Минимум 1 источник волн"
assert 0<= R <= w and 0<= R <= h, "НЕ верный размер окружности"
T = 100
images = []
x_C = w/2
y_C = h/2
for t_i in range(T):
  X = np.tile(np.linspace(0,w,w), (w, 1))
  Y = np.tile(np.linspace(0,h,h).reshape((-1,1)), (1, h))
  pixels = np.full((w,h,3), 255)
  
  pixels[:,:,0] = pixels[:,:,1] = pixels[:,:,2] = sum([phases_waves(t=t_i,x=X,y=Y,x0=x_C + R * np.cos(t_i+i*np.pi/360*N),y0=y_C+R * np.sin(t_i+i*np.pi/360*N)) for i in range(N)])
  
  image = Image.fromarray(pixels.astype('uint8'), 'RGB')
  images.append(image)
  image.save(f"{WAVES_IMAGES}/{t_i}.jpeg")
  
imageio.mimsave(f'{WAVES_GIFS}/waves_from_{N}_sorces.gif', images)


Количество источников: 5
Радиус круговой траектории: 125.7
